### **여러가지 LLM을 사용한 Tool Calling Agent**

`Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원

In [1]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews


# 도구 정의
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

# tools 정의
tools = [search_news]

도구 이름: search_news
도구 설명: Search Google News by input keyword


### **Agent Prompts**

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# # Llama-3.1-70B-Instruct-Turbo
# llama = ChatOpenAI(
#     base_url="https://api.together.xyz/v1",
#     api_key=os.environ["TOGETHER_API_KEY"],
#     model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
# )

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# Qwen2.5 7B (한글 성능 괜찮은 편)
qwen = ChatOllama(
    model="qwen2.5:7b",
)

In [10]:
from langchain.agents import create_tool_calling_agent

# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
# llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=qwen_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 投资'}`


[{'url': 'https://news.google.com/rss/articles/CBMia0FVX3lxTE1IdjVHZDF4S1Qzc2w1QnNpVC10U0hhY1M0UnJET1hMdmdCeFJmZXR0RHJaX1VxR1hQNmhsV3p5T052NTZWRm9PZGtEX0hleUdmZktEd2gtUm02YllKNzhGdGlNTm9HRjFsdDBB?oc=5', 'content': '中 휴머노이드 로봇 기업 푸리에, 1500억원 투자 유치 - 로봇신문사'}, {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBTb0RYWF9pcjQ5dkhLMVdobDFGZmstU0RkSGM0S2gtdEo4RVpaUGRuQlpyaUd0S1haRjNyYkx1aHYyM25HSjVpaHVWMEpkei1zOU1CVndB?oc=5', 'content': '비트코인이 안전자산? 가격과 명성에서 금과 경쟁… VVIP ‘안전자산 포트폴리오’에 관심 [AI 프리즘*금융상품 투자자 뉴스] - 서울경제'}, {'url': 'https://news.google.com/rss/articles/CBMiVEFVX3lxTFA1N3BGTnBFWk0xS0hpRlZUNnR1UDdfaEJoM0ZRYU5fWk83dUtibVdTQUpMc2tjMXJMcnZDZWJDVDdEUW1Pakowb3N2TjlCWkhpVW5ibA?oc=5', 'content': '엔비디아 CEO "AI 에이전트 수조 달러 산업 될 것" 전망 - BeInCrypto Korea'}, {'url': 'https://news.google.com/rss/articles/CBMiTEFVX3lxTE4xRjlaX3JlajFQMU9YTEZmUzVJMDNNWTRVc3lPMFYwczE2MFFKcTdwTzBOU1E1YllfTnlGT25sTkRKVER

In [12]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

In [13]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

# gpt
execute_agent(gpt, tools, query, "gemini")

[gemini] 결과입니다.
🌟 **AI 투자 소식** 🌟

🚀 최근 AI 분야의 투자 열기가 뜨겁습니다! 📈

1️⃣ **오픈AI**, 무려 **400억 달러** 투자를 유치하며 최근 5개월 사이에 기업 가치를 두 배로 끌어올렸습니다! [자세히 보기](https://news.google.com/rss/articles/CBMiU0FVX3lxTE5DaDRFcGh4Z25NZDZKS3ZkSXFMOGs0RmkzcEFJZlBMaGVCVHJRZlV6b1NnVk9ydWQ0eVBNZFBQOUlMY05CY0hCTVZpLXlkTEFrWUJ3?oc=5) 🔥

2️⃣ 투자 앱 **핀트**가 미국에서 프리미엄 투자전략 상품 출시를 발표했어요! [더 알아보기](https://news.google.com/rss/articles/CBMiW0FVX3lxTE9OcFFfVE1VcWxWaEdsQV8xV19tcnRlUTdXTDRVdy0tWHRiOTVybVoyQjRORGVLbGpDbVNteElrTS1EU3ozMzJvZHJXSUNaVmxnOVhncFZGd2xjelHSAWBBVV95cUxOdEY2NWlxejFSWjgwU3FPakhkcTNyUHVFRURTY2JMcTVXSjFzcnZPSXdieS05N19LV1BRS1U5RkRfM05ORUNMeW5iVjVEVWZ1cVZtZlp0c3h0QUhrcEJqQlk?oc=5) 💼

3️⃣ **뤼튼테크놀로지스**가 1080억원 규모의 시리즈B 투자 유치를 완료했습니다! [눈여겨보세요](https://news.google.com/rss/articles/CBMiakFVX3lxTE8xeTBoWWYwaFJCRFBNXzgyXzN0V3hyM1VjNlVVYzJZaDRGQndRUmpMeFR5bGJTdlZwWFRrYVkwMXpnMUxSQnRXM0k3cWp3UWY5YXlMZHoxdmNxXzhHNTlPQi1md0hXc3VqYXc?oc=5) 📊

4️⃣ 정부가 AI 인프라를 구축하면 기업들이 AI 투자에 나설 것이라는 전망도 나왔습니다. 중국의 자율주행 상용화에 박차를 가할 예정입니다. [더